In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.models.detection import retinanet_resnet50_fpn_v2
from torchvision.models.detection import RetinaNet_ResNet50_FPN_V2_Weights
from torchvision.transforms import v2
from torchvision.io import read_image
from torchvision.utils import save_image

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
model = retinanet_resnet50_fpn_v2(weights=RetinaNet_ResNet50_FPN_V2_Weights.DEFAULT)
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model.to(device)

In [4]:
input_dir = '/home/scs/taylor/faculty/onlySD312'
output_dir = input_dir+'/cropped'

In [5]:
transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize(size=(800,800),antialias=True)
])

In [7]:
for filename in os.listdir(input_dir):
    if filename.endswith('.jpg'):
        image=read_image(input_dir+'/'+filename)
        image_tensor = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            predictions = model(image_tensor)
            #print(predictions)
        for idx, prediction in enumerate(predictions[0]['boxes']):
            box = prediction.cpu().numpy().astype(int)
            x1, y1, x2, y2 = box
            cropped_face = image[:, y1:y2, x1:x2]
            print(cropped_face, cropped_face.shape)

            # Save the cropped face to a file
            output_filename = f'{os.path.splitext(filename)[0]}_face{idx+1}.jpg'
            output_path = os.path.join(output_dir, output_filename)

            permuted_tensor = cropped_face.unsqueeze(0).permute(0, 1, 2, 3)

            normalized_tensor = permuted_tensor / 255
            save_image(normalized_tensor,output_path)

tensor([[[120, 122, 122,  ..., 139, 140, 140],
         [121, 122, 123,  ..., 139, 139, 140],
         [119, 121, 123,  ..., 139, 139, 139],
         ...,
         [  9,   9,   9,  ...,   5,   6,  10],
         [  9,   9,   9,  ...,   5,   6,  10],
         [  9,   9,   9,  ...,   5,   6,  10]],

        [[149, 151, 151,  ..., 163, 164, 164],
         [150, 151, 152,  ..., 163, 163, 164],
         [148, 150, 152,  ..., 163, 163, 163],
         ...,
         [  9,   9,   9,  ...,   5,   6,  10],
         [  9,   9,   9,  ...,   5,   6,  10],
         [  9,   9,   9,  ...,   5,   6,  10]],

        [[179, 181, 181,  ..., 189, 190, 190],
         [180, 181, 182,  ..., 189, 189, 190],
         [178, 180, 182,  ..., 189, 189, 189],
         ...,
         [  9,   9,   9,  ...,   5,   6,  10],
         [  9,   9,   9,  ...,   5,   6,  10],
         [  9,   9,   9,  ...,   5,   6,  10]]], dtype=torch.uint8) torch.Size([3, 320, 320])
tensor([], size=(3, 0, 302), dtype=torch.uint8) torch.Size([3

ValueError: cannot write empty image as JPEG